In [1]:
import geopandas as gpd
import pandas as pd
import folium
from branca.colormap import linear
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import openmatrix as omx

In [2]:
# List of TAZ rows to extract
taz_rows_to_extract_df = pd.DataFrame([
    [ 832, 'Bountiful'    ],
    [1629, 'Murray'       ],
    [1984, 'South Jordan' ],
    [3167, 'Spanish Fork' ],
    [2845, 'MAG Office'   ],
    [2513, 'Saratoga'      ]
], columns=['taz_id','map_label'])

taz_rows_to_extract = taz_rows_to_extract_df['taz_id'].drop_duplicates().to_list()

output_csv = "intermediate/TAZ-PA-Time.csv"

# Define the input OMX file and output CSV file
omx_files_df = pd.DataFrame([
    ['auto', 'data/input_matrix_auto.omx'],
    ['transit', 'data/input_matrix_transit.omx']
], columns=['AutoTransit','omx_filename'])
display(omx_files_df)

matrices_for_extraction_df = pd.DataFrame([
    ['New','auto','none',['GP_TotTime']],#,'MG_TotTime']],
    ['Old','auto','none',['GP_TotTime']],#,'MG_TotTime']],
    ['New','transit','walk',['w4time','w5time','w6time','w7time','w8time','w9time']],
    ['New','transit','drive',['d4time','d5time','d6time','d7time','d8time','d9time']],
    ['Old','transit','both',['w4time','w5time','w6time','w7time','w8time','w9time','d4time','d5time','d6time','d7time','d8time','d9time']],
], columns=['Method','AutoTransit','AccessMode','matrices_to_get_minimum'])
display(matrices_for_extraction_df)

,AutoTransit,omx_filename
0,auto,data/input_matrix_auto.omx
1,transit,data/input_matrix_transit.omx


,Method,AutoTransit,AccessMode,matrices_to_get_minimum
0,New,auto,none,[GP_TotTime]
1,Old,auto,none,[GP_TotTime]
2,New,transit,walk,"[w4time, w5time, w6time, w7time, w8time, w9time]"
3,New,transit,drive,"[d4time, d5time, d6time, d7time, d8time, d9time]"
4,Old,transit,both,"[w4time, w5time, w6time, w7time, w8time, w9tim..."


In [4]:
# Convert TAZ rows to zero-based index
matrix_rows_to_extract = [i - 1 for i in taz_rows_to_extract]

# Initialize list to store extracted data
extracted_data = []

# Loop through OMX files
for _, row_files in omx_files_df.iterrows():
    auto_transit = row_files['AutoTransit']

    # Open OMX file
    with omx.open_file(row_files['omx_filename'], 'r') as omx_data:
        
        # Filter matrices for the current AutoTransit mode
        matrices_for_extraction_filtered_df = matrices_for_extraction_df[
            matrices_for_extraction_df['AutoTransit'] == auto_transit
        ]

        # Loop over each AutoTransit & AccessMode combination
        for _, row in matrices_for_extraction_filtered_df.iterrows():
            method = row["Method"]
            access_mode = row["AccessMode"]
            matrices_to_extract = row["matrices_to_get_minimum"]

            # Extract and compute the minimum values across matrices for each I-J pair
            for i in matrix_rows_to_extract:
                for j in range(omx_data[matrices_to_extract[0]].shape[1]):  # Iterate over all columns (J values)
                    
                    # Extract values across all matrices at (i, j), filtering out 0 values
                    matrix_values = [omx_data[matrix][i, j] for matrix in matrices_to_extract]
                    non_zero_values = [val for val in matrix_values if val > 0]  # Filter out 0s
                    
                    # Get the minimum value across non-zero matrices (if any remain)
                    min_value = min(non_zero_values) if non_zero_values else None
                    
                    # Append extracted data only if there's a valid minimum
                    if min_value is not None:
                        extracted_data.append([i + 1, j + 1, min_value, auto_transit, access_mode, method])

# Convert to DataFrame
extracted_data_df = pd.DataFrame(extracted_data, columns=['TAZ_P', 'TAZ_A', 'Value', 'AutoTransit', 'AccessMode', 'Method'])
display(extracted_data_df)

,TAZ_P,TAZ_A,Value,AutoTransit,AccessMode,Method
0,832,1,77.70,auto,none,New
1,832,2,77.14,auto,none,New
2,832,3,72.45,auto,none,New
3,832,4,73.51,auto,none,New
4,832,5,76.02,auto,none,New
...,...,...,...,...,...,...
68116,2513,3455,92.05,transit,both,Old
68117,2513,3460,160.12,transit,both,Old
68118,2513,3461,94.36,transit,both,Old
68119,2513,3462,153.61,transit,both,Old


In [10]:
extracted_data_df[(extracted_data_df['Method']=='Old') & (extracted_data_df['AccessMode']=='both')]

,TAZ_P,TAZ_A,Value,AutoTransit,AccessMode,Method
58419,832,19,171.52,transit,both,Old
58420,832,20,166.42,transit,both,Old
58421,832,24,168.84,transit,both,Old
58422,832,25,160.35,transit,both,Old
58423,832,26,158.06,transit,both,Old
...,...,...,...,...,...,...
68116,2513,3455,92.05,transit,both,Old
68117,2513,3460,160.12,transit,both,Old
68118,2513,3461,94.36,transit,both,Old
68119,2513,3462,153.61,transit,both,Old


In [5]:
# Save to CSV
extracted_data_df.to_csv(output_csv, index=False)

print(f"Extracted data saved to {output_csv}")

Extracted data saved to intermediate/TAZ-PA-Time.csv


In [6]:
travel_shed_ato_classes_df = pd.read_csv("params/NEW And OLD ATO Method (for travelsheds).csv")
travel_shed_ato_classes_df

,Method,AutoTransit,AccessMode,TTStart_Minutes,TTEnd_Minutes_Inclusive,MapClass,JobFract
0,New,auto,none,0.00,10.00,11,1
1,New,auto,none,10.00,13.50,10,0.90 - 0.99
2,New,auto,none,13.50,17.00,9,0.80 - 0.89
3,New,auto,none,17.00,20.50,8,0.70 - 0.79
4,New,auto,none,20.50,24.00,7,0.60 - 0.69
5,New,auto,none,24.00,27.50,6,0.50 - 0.59
6,New,auto,none,27.50,31.00,5,0.40 - 0.49
7,New,auto,none,31.00,34.50,4,0.30 - 0.39
8,New,auto,none,34.50,38.00,3,0.20 - 0.29
9,New,auto,none,38.00,41.50,2,0.10 - 0.19


In [7]:
# Performing the join based on the Value being between TTStart_Minutes and TTEnd_Minutes
merged_df = extracted_data_df.merge(
    travel_shed_ato_classes_df,
    on=['Method','AutoTransit','AccessMode']
).query("TTStart_Minutes < Value <= TTEnd_Minutes_Inclusive")

# Display the merged DataFrame
display(merged_df)


,TAZ_P,TAZ_A,Value,AutoTransit,AccessMode,Method,TTStart_Minutes,TTEnd_Minutes_Inclusive,MapClass,JobFract
6335,832,576,44.47,auto,none,New,41.50,45.0,1,0.00 - 0.09
6555,832,596,44.81,auto,none,New,41.50,45.0,1,0.00 - 0.09
6610,832,601,43.02,auto,none,New,41.50,45.0,1,0.00 - 0.09
6830,832,621,44.03,auto,none,New,41.50,45.0,1,0.00 - 0.09
6885,832,626,44.03,auto,none,New,41.50,45.0,1,0.00 - 0.09
...,...,...,...,...,...,...,...,...,...,...
721100,2513,3020,55.30,transit,both,Old,39.25,60.0,1,0.00 - 0.09
721111,2513,3021,56.78,transit,both,Old,39.25,60.0,1,0.00 - 0.09
721122,2513,3022,57.97,transit,both,Old,39.25,60.0,1,0.00 - 0.09
721177,2513,3027,57.69,transit,both,Old,39.25,60.0,1,0.00 - 0.09


In [12]:
# Load the TAZ GeoJSON file
taz_geo = gpd.read_file("taz/WFv910_TAZ.geojson")
taz_df = pd.read_csv("taz/WFv910_TAZ.csv")
taz_geo_merged = pd.merge(taz_geo, taz_df, on="TAZID")
taz_geo_merged = taz_geo_merged[(taz_geo_merged['REMM']==1) & (taz_geo_merged['DEVPBLEPCT']>0)]
taz_geo = taz_geo_merged
#display(taz_geo_merged)

# Merge TAZ data with the extracted dataset using 'TAZID'
merged_geo = taz_geo.merge(merged_df, left_on="TAZID", right_on="TAZ_A")

# Set up Chrome WebDriver for capturing PNGs
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1200x800")
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Custom RGB color palette for MapClass (converted to HEX)
mapclass_colors = {
    11: "#ACFA70",  # (172,250,112)
    10: "#70EA80",  # (112,234,128)
    9:  "#23D890",  # (35,216,144)
    8:  "#00C49E",  # (0,196,158)
    7:  "#00ADA4",  # (0,173,164)
    6:  "#0097A3",  # (0,151,163)
    5:  "#00829D",  # (0,130,157)
    4:  "#006D95",  # (0,109,149)
    3:  "#005886",  # (0,88,134)
    2:  "#204370",  # (32,67,112)
    1:  "#292F56",  # (41,47,86)
}

# Function to create a Folium map with custom colors and matching polygon borders
def create_map(geo_data, title, center, zoom_level, taz_p_location=None):
    """Creates a Folium map with a custom color scheme, polygon borders, and a marker at TAZ_P."""
    map_obj = folium.Map(location=taz_p_location, zoom_start=zoom_level)

    if geo_data.empty or geo_data["geometry"].isna().all():
        # If no data, add a marker indicating "No Data Available"
        folium.Marker(
            location=center,
            popup="No Data Available",
            icon=folium.Icon(color="gray")
        ).add_to(map_obj)
    else:
        # Style function to apply custom colors and matching border color
        def style_function(feature):
            mapclass = feature["properties"].get("MapClass", None)
            fill_color = mapclass_colors.get(mapclass, "#FFFFFF")  # Default to white if not found
            return {
                "fillColor": fill_color,
                "color": fill_color,  # **MATCH BORDER TO FILL COLOR**
                "weight": 0.5,  # Slight weight to avoid slivers
                "fillOpacity": 0.85
            }

        folium.GeoJson(
            geo_data,
            style_function=style_function,
            tooltip=folium.GeoJsonTooltip(fields=["MapClass"])
        ).add_to(map_obj)

    # Add a marker for the TAZ_P location if available
    if taz_p_location:
        folium.Marker(
            location=taz_p_location,
            popup=f"TAZ_P: {title}",
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(map_obj)

    return map_obj


# Folder to store PNGs
output_folder = "output_maps"
os.makedirs(output_folder, exist_ok=True)

# Loop through each unique I value to create separate maps
for _, row_taz_rows_to_extract_df in taz_rows_to_extract_df.iterrows():

    taz_p_val = row_taz_rows_to_extract_df['taz_id']
    map_label = row_taz_rows_to_extract_df['map_label']

    for _, row in matrices_for_extraction_df.iterrows():

        auto_transit = row['AutoTransit']
        access_mode = row['AccessMode']

        subset_geo = merged_geo[
            (merged_geo["TAZ_P"] == taz_p_val) &
            (merged_geo['AutoTransit'] == auto_transit) &
            (merged_geo['AccessMode'] == access_mode)
        ]

        # Separate datasets for New and Old methods
        new_geo = subset_geo[subset_geo["Method"] == "New"]
        old_geo = subset_geo[subset_geo["Method"] == "Old"]

        # Define a consistent zoom extent based on the full dataset for `TAZ_P`
        center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
        zoom_level = 10  # Adjust if necessary

        # Find the centroid of the TAZ polygon where TAZID == TAZ_P
        taz_p_location = None
        taz_match = taz_geo[taz_geo["TAZID"] == taz_p_val]

        if not taz_match.empty:
            taz_p_location = [
                taz_match.geometry.centroid.y.values[0],
                taz_match.geometry.centroid.x.values[0]
            ]

        # Ensure an empty GeoDataFrame is properly structured
        empty_geojson = gpd.GeoDataFrame({'MapClass': [], 'geometry': []}, geometry='geometry', crs="EPSG:4326")

        # Create maps with the correct TAZ marker
        if not new_geo.empty:
            new_map = create_map(new_geo, f"{map_label}_{auto_transit}_{access_mode}_New_Method", center, zoom_level, taz_p_location)
            new_map_html = f"output_html/map_temp_{map_label}_{auto_transit}_{access_mode}_new.html"
            new_map.save(new_map_html)
        else:
            print(f"No data for New Method: {map_label}, {auto_transit}, {access_mode}")

        if not old_geo.empty:
            old_map = create_map(old_geo, f"{map_label}_{auto_transit}_{access_mode}_Old_Method", center, zoom_level, taz_p_location)
            old_map_html = f"output_html/map_temp_{map_label}_{auto_transit}_{access_mode}_old.html"
            old_map.save(old_map_html)
        else:
            print(f"No data for Old Method: {map_label}, {auto_transit}, {access_mode}")

        # Save maps as temporary HTML files
        

        # Convert HTML to PNG
        def save_as_png(html_file, output_file):
            """Renders the HTML map and saves it as a PNG image."""
            file_url = f"file://{os.path.abspath(html_file)}"
            driver.get(file_url)
            driver.save_screenshot(output_file)
            print(f"Saved: {output_file}")

        # File names with TAZ_P at the beginning
        if not new_geo.empty:
            if access_mode == 'none' or access_mode == 'both':
                new_png = os.path.join(output_folder, f"{map_label}_{auto_transit}_New_Method.png")
            else:
                new_png = os.path.join(output_folder, f"{map_label}_{auto_transit}_{access_mode}_New_Method.png")
            save_as_png(new_map_html, new_png)
            os.remove(new_map_html)
        if not old_geo.empty:
            if access_mode == 'none' or access_mode == 'both':
                old_png = os.path.join(output_folder, f"{map_label}_{auto_transit}_Old_Method.png")
            else:
                old_png = os.path.join(output_folder, f"{map_label}_{auto_transit}_{access_mode}_Old_Method.png")
            save_as_png(old_map_html, old_png)
            os.remove(old_map_html)

# Close the WebDriver
driver.quit()

print(f"All PNG maps saved in '{output_folder}' folder.")


C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:111: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.y.values[0],
C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.x.values[0]


Saved: output_maps\Bountiful_auto_New_Method.png
Saved: output_maps\Bountiful_auto_Old_Method.png


C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:111: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.y.values[0],
C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.x.values[0]


Saved: output_maps\Bountiful_auto_New_Method.png
Saved: output_maps\Bountiful_auto_Old_Method.png
No data for Old Method: Bountiful, transit, walk


C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:111: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.y.values[0],
C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.x.values[0]


...

Saved: output_maps\Saratoga_transit_walk_New_Method.png
No data for Old Method: Saratoga, transit, drive


C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:111: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.y.values[0],
C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.x.values[0]


Saved: output_maps\Saratoga_transit_drive_New_Method.png
No data for New Method: Saratoga, transit, both


C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:111: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.y.values[0],
C:\Users\bhereth\AppData\Local\Temp\ipykernel_4044\3425084844.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.x.values[0]


Saved: output_maps\Saratoga_transit_Old_Method.png
All PNG maps saved in 'output_maps' folder.
